In [1]:
import numpy as np
import pandas as pd

In [73]:
a, b = 1.5, 2.0
# a, b = 0.6, 1.1

i = 10
h = (b - a) / i

x_points = np.array([1.52, 1.97, 1.77])

def y(x: float) -> float:
    """
    Function to compute the given mathematical expression.

    Args:
    x (float): Input value

    Returns:
    float: Computed result
    """
    return np.pow(x, 2) + np.log(x) - 4

x_val = np.arange(a, a + h * (i + 1), h)
y_val = y(x_val)
x_val, y_val

(array([1.5 , 1.55, 1.6 , 1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  ]),
 array([0.25185389, 0.47032937, 0.69314718]))

In [101]:
def diff_y(x: float) -> float:
    """
    Function to compute the given mathematical expression.

    Args:
    x (float): Input value

    Returns:
    float: Computed result
    """
    return 2 * x + 1 / x

y_diff = diff_y(x_val)
y_diff

array([3.66666667, 3.74516129, 3.825     , 3.90606061, 3.98823529,
       4.07142857, 4.15555556, 4.24054054, 4.32631579, 4.41282051,
       4.5       ])

## Численное дифференцирование по узлу 2 для полинома Лагранжа 3 степени

Дана таблично заданная функция в узлах $x_1, x_2, x_3, x_4$. Требуется найти производную в точке $x_2$.

Полином Лагранжа 3 степени:
$$ L_3(x) = \sum_{i=1}^4 f(x_i) \cdot \ell_i(x) $$

где базисные полиномы:
$$ \ell_i(x) = \prod_{\substack{j=1 \\ j \neq i}}^4 \frac{x - x_j}{x_i - x_j} $$

### Производная полинома Лагранжа
$$ L_3'(x) = \sum_{i=1}^4 f(x_i) \cdot \ell_i'(x) $$

В точке $x_2$:
$$ f'(x_2) \approx L_3'(x_2) = \sum_{i=1}^4 f(x_i) \cdot \ell_i'(x_2) $$

### Вычисление производных базисных полиномов в $x_2$

Для узлов $x_1, x_2, x_3, x_4$:

1. $\ell_1'(x_2) = \frac{(x_2-x_3)(x_2-x_4) + (x_2-x_1)(x_2-x_4) + (x_2-x_1)(x_2-x_3)}{(x_1-x_2)(x_1-x_3)(x_1-x_4)}$

2. $\ell_2'(x_2) = \frac{1}{x_2-x_1} + \frac{1}{x_2-x_3} + \frac{1}{x_2-x_4}$

3. $\ell_3'(x_2) = \frac{(x_2-x_1)(x_2-x_4) + (x_2-x_1)(x_2-x_2) + (x_2-x_2)(x_2-x_4)}{(x_3-x_1)(x_3-x_2)(x_3-x_4)}$

4. $\ell_4'(x_2) = \frac{(x_2-x_1)(x_2-x_2) + (x_2-x_1)(x_2-x_3) + (x_2-x_2)(x_2-x_3)}{(x_4-x_1)(x_4-x_2)(x_4-x_3)}$

### Для равномерной сетки (шаг $h$)

Пусть $x_i = x_1 + (i-1) \cdot h$, тогда:

1. $\ell_1'(x_2) = \frac{(-h)(-2h) + (h)(-2h) + (h)(-h)}{(-h)(-2h)(-3h)} = \frac{2h^2 - 2h^2 - h^2}{-6h^3} = \frac{-h^2}{-6h^3} = \frac{1}{6h}$

2. $\ell_2'(x_2) = \frac{1}{h} + \frac{1}{-h} + \frac{1}{-2h} = -\frac{1}{2h}$

3. $\ell_3'(x_2) = \frac{(h)(-2h) + (h)(0) + (0)(-2h)}{(2h)(h)(-h)} = \frac{-2h^2}{-2h^3} = \frac{1}{h}$

4. $\ell_4'(x_2) = \frac{(h)(0) + (h)(-h) + (0)(-h)}{(3h)(2h)(h)} = \frac{-h^2}{6h^3} = -\frac{1}{6h}$

### Итоговая формула

$$ f'(x_2) \approx \frac{1}{6h}f(x_1) - \frac{1}{2h}f(x_2) + \frac{1}{h}f(x_3) - \frac{1}{6h}f(x_4) $$

Упростив, получаем:

$$ f'(x_2) \approx \frac{f(x_1) - 3f(x_2) + 6f(x_3) - 2f(x_4)}{6h} $$



In [105]:
def first_diff_dn(
        node_num: int, 
        x_values: np.ndarray, 
        y_values: np.ndarray, 
        interpolation_deg: int
) -> tuple[float, np.ndarray, np.ndarray]:
    """
    Compute the numerical first derivative at a given node using Lagrange polynomial interpolation for equal intervals between nodes.

    This function approximates the derivative at `node_num` by fitting a polynomial of degree 
    `interpolation_deg` to nearby points and analytically differentiating it. The method is 
    particularly useful for non-uniform grids (though uniform grids are more stable).

    Parameters
    ----------
    node_num : int
        Index of the node where the derivative is computed (0-based indexing).
    x_values : np.ndarray
        Array of x-coordinates (grid nodes). Must be sorted in ascending order.
    y_values : np.ndarray
        Array of function values at `x_values`.
    interpolation_deg : int
        Degree of the interpolating polynomial. The number of points used is `interpolation_deg + 1`.

    Returns
    -------
    tuple[float, np.ndarray, np.ndarray]
        - Approximate derivative at `x_values[node_num]`.
        - Subset of `x_values` used for interpolation (shape: `(interpolation_deg + 1)`).
        - Coefficients of the differentiation formula (for debugging/analysis).
    """
    
    if len(x_values) < interpolation_deg + 1:
        raise ValueError("Node quantity must be greater than interpolation_deg + 1.")
    if node_num > len(x_values) - 1:
        raise ValueError("Node must be less than the number of nodes.")
    
    if node_num - (interpolation_deg + 1) // 2 < 0:
        start = 0
        end = interpolation_deg + 1
    elif node_num + (interpolation_deg + 1) // 2 > len(x_values):
        start = len(x_values) - (interpolation_deg + 1)
        end = len(x_values)
    else: 
        start = node_num - (interpolation_deg + 1) // 2
        end = node_num + (interpolation_deg + 1) // 2 + 1 if (interpolation_deg + 1) % 2 != 0 else node_num + (interpolation_deg + 1) // 2

    nodes = x_values[start:end]
    values = y_values[start:end]
    
    h = x_values[1] - x_values[0]
    coefficients = np.array([])
    D = 0
    
    for i in range(interpolation_deg + 1):
        d = 0
        for j in range(interpolation_deg + 1):
            if i == j:
                continue
            d += np.prod([x_values[node_num] - nodes[k] for k in range(interpolation_deg + 1) if k != i and k != j])
        
        coefficient = d / (np.prod([i - l for l in range(interpolation_deg + 1) if l != i]))
        coefficients = np.append(coefficients, coefficient)
        D += coefficient * values[i]
        
    D = D / np.pow(h, interpolation_deg)
    coefficients = coefficients / np.pow(h, interpolation_deg)
    
    return float(D), nodes ,coefficients

### Оценим сверху и снизу погрешность численного дифференцирования 3-го порядка. Для этого используем выражение для остаточного члена интерполяционного многочлена:

$$
R_2(x) = \frac{f^{(4)}(\xi) \cdot \omega_{4}(x)}{4!}, \quad \xi \in [1.5, 1.65]
$$

где

$$
\omega_4(x) = (x - x_0)(x - x_1)(x - x_2)(x - x_3)
$$

а функция задана как:

$$
f(x) = x^2 + \ln(x) - 4
$$

Тогда производные:

$$
f'(x) = 2x + \frac{1}{x}, \quad
f''(x) = 2 - \frac{1}{x^2}, \quad
f^{(3)}(x) = \frac{2}{x^3}, \quad
f^{(4)}(x) = -\frac{6}{x^4}
$$

На отрезке $[1.5, 1.65]$ четвёртая производная ограничена:

$$
\min f^{(4)}(x) = -\frac{6}{(1.65)^4} \approx -\frac{6}{7.398} \approx -0.811
$$

$$
\max f^{(4)}(x) = -\frac{6}{(1.5)^4} = -\frac{6}{5.0625} \approx -1.185
$$

Следовательно, на отрезке $[1.5, 1.65]$:

$$
f^{(4)}(x) \in [-1.185, -0.811]
$$

Для оценки $\omega_4(x)$ выберем точку $x = 1.575$ (центр отрезка). Тогда:

$$
\omega_4(1.575) = (1.575 - 1.5)(1.575 - 1.55)(1.575 - 1.6)(1.575 - 1.65) = (0.075)(0.025)(-0.025)(-0.075)
$$

$$
= (0.075 \cdot -0.075)(0.025 \cdot -0.025) = (-0.005625)(-0.000625) = 3.515625 \cdot 10^{-6}
$$

Теперь можем найти двустороннюю оценку погрешности:

$$
\frac{|\min f^{(4)}(\xi)| \cdot |\omega_4(x)|}{4!} \leq |R_2(x)| \leq \frac{|\max f^{(4)}(\xi)| \cdot |\omega_4(x)|}{4!}
$$

$$
\frac{0.811 \cdot 3.515625 \cdot 10^{-6}}{24} \leq |R_2(x)| \leq \frac{1.185 \cdot 3.515625 \cdot 10^{-6}}{24}
$$

Посчитаем границы:

$$
\text{Нижняя граница: } \frac{0.811 \cdot 3.515625 \cdot 10^{-6}}{24} \approx 1.188 \cdot 10^{-7}
$$

$$
\text{Верхняя граница: } \frac{1.185 \cdot 3.515625 \cdot 10^{-6}}{24} \approx 1.735 \cdot 10^{-7}
$$

Итоговая двусторонняя оценка погрешности:

$$
\boxed{
1.19 \cdot 10^{-7} \leq |R_2(x)| \leq 1.74 \cdot 10^{-7}
}
$$


In [159]:
node = 2
deg = 3

d_2 = first_diff_dn(node, x_val, y_val, deg)

df1 = pd.DataFrame({'Вычисленная производная': [d_2[0]], 'Истинное значение': [y_diff[node]], 'Ошибка':  [d_2[0] - y_diff[node]]})
df2 = pd.DataFrame({'Узел':d_2[1], 'Коэффициент при узле': d_2[2]})

print("Результаты численного дифференцирования:")
display(df1)

print("\nКоэффициенты полинома Лагранжа:")
display(df2)

Результаты численного дифференцирования:


,Вычисленная производная,Истинное значение,Ошибка
0,3.82499,3.825,-0.00001



Коэффициенты полинома Лагранжа:


,Узел,Коэффициент при узле
0,1.50,3.333333
1,1.55,-20.000000
2,1.60,10.000000
3,1.65,6.666667


### Ошибка получилась чуть больше теоретической, но все равно достаточно мала, для того, чтобы утвержать о корректности формулы

### Так как функция для вычисления является уникальной, можно отследить как от увеличения количества узлов повышается точность приближения:

In [160]:
node = 5
d_2 = []
for deg in range(2,11):
    d_2.append(first_diff_dn(node, x_val, y_val, deg)[0])
df1 = pd.DataFrame({'Степень полинома': range(2, 11), 'Вычисленная производная': d_2, 'Истинное значение': [y_diff[node]] * len(d_2), 'Ошибка':  np.array(d_2) - y_diff[node]})
print("Пронаблюдаем точность приближения производной для узла 5 сетки:")
df1

Пронаблюдаем точность приближения производной для узла 5 сетки:


,Степень полинома,Вычисленная производная,Истинное значение,Ошибка
0,2,4.071584,4.071429,1.555670e-04
1,3,4.071422,4.071429,-6.987595e-06
2,4,4.071428,4.071429,-3.055255e-07
3,5,4.071429,4.071429,2.356042e-08
4,6,4.071429,4.071429,1.612984e-09
5,7,4.071429,4.071429,-1.796083e-10
6,8,4.071429,4.071429,-1.656542e-11
7,9,4.071429,4.071429,2.454037e-12
8,10,4.071429,4.071429,2.877698e-13
